In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import gensim
import string
import re
#import vis_topic as vt
import corex_topic as ct
import scipy.sparse as ss
import matplotlib.pyplot as plt

In [ ]:
# read .txt file line by line, split by tab(\t)
list_data=[]
with open('C:\\Users\\Gaurav\\Desktop\\ISI forms\\Project\\il5_uromo.txt',encoding='utf8',errors='ignore') as fp:
    for line in fp:
        list_data.append(line.split('\t'))
    
    

In [ ]:

    # name the columns of datframe
raw_data = pd.DataFrame(list_data,columns=['doc_id','text_data','class_type']) 

In [ ]:
# strip columns for leading and trailing white spaces
raw_data['doc_id']=raw_data.doc_id.str.strip()
raw_data['text_data']=raw_data.text_data.str.strip()
raw_data['class_type']=raw_data['class_type'].str.strip()

In [ ]:
raw_data.shape

In [ ]:
# set doc_id as index
raw_data= raw_data.set_index('doc_id')
# change "class_type" column to categorical datatype
raw_data['class_type'] = raw_data['class_type'].astype('category')

In [ ]:
# http links removal on 'text_data' column

# regex : ((http|https)://t.co/[a-zA-Z0-9]+)

raw_data['text_data'] = raw_data['text_data'].apply(lambda x: re.sub('((http|https)://t.co/[a-zA-Z0-9]+)','',x))
raw_data.shape

In [ ]:
# RT (Retweet Removal)

raw_data['text_data'] = raw_data['text_data'].apply(lambda x: re.sub('RT','',x))
raw_data.shape

In [ ]:
#punctuation removal on 'text_data' column
#print(string.punctuation)
punct='!"$%&()*+,-./:;<=>?[\]^_`{|}~'+"'"
#print(punct)
regex = re.compile('[%s]' % re.escape(punct))

raw_data['text_data'] = raw_data['text_data'].apply(lambda x: regex.sub('', x))
raw_data['text_data']=raw_data.text_data.str.strip()
raw_data.shape

In [ ]:
# remove @names

raw_data['text_data'] = raw_data['text_data'].apply(lambda x: re.sub('\@[a-zA-Z0-9]+','',x))
raw_data.shape

In [ ]:
raw_data['length'] = raw_data['text_data'].apply(lambda x: len(x.split()))
# sort by file length
raw_data=raw_data.sort_values(by='length', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [ ]:
raw_data = raw_data.loc[raw_data.length>0]
raw_data.head(5)

In [ ]:
# remove emoji's

raw_data['text_data'] = raw_data['text_data'].apply(lambda x: re.sub('["\U0001F600-\U0001F64F" "\U0001F300-\U0001F5FF" "\U0001F680-\U0001F6FF"  "\U0001F1E0-\U0001F1FF"]+',' ',x))

# strip whitespaces again 


raw_data['text_data']=raw_data.text_data.str.strip()
raw_data['class_type']=raw_data['class_type'].str.strip()

# calculate length again
raw_data['length'] = raw_data['text_data'].apply(lambda x: len(x.split()))
# sort by file length
raw_data=raw_data.sort_values(by='length', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [ ]:
raw_data = raw_data.loc[raw_data.length>5]
raw_data.head(5)

In [ ]:
raw_data['text_data'] = raw_data['text_data'].apply(lambda x: x.lower())

In [ ]:
print(raw_data.class_type.value_counts())
plt.hist('class_type',data=raw_data)
plt.show()

In [ ]:
# write the formatted data to a new text file for reference
#print(raw_data.to_csv(sep='\t', index=False, header=True)) 
reset_data=raw_data.reset_index()
reset_data.to_csv('C:\\Users\\Gaurav\\Desktop\\ISI forms\\Project\\il5_data__preprocess_hashtag_emojis_length.txt', header=True, index=False, sep='\t', mode='w',encoding='utf-8')


In [ ]:
count =raw_data.length.value_counts()
count = count.reset_index()

In [ ]:
count=count.rename(index=str, columns={'index':'Document Length','length':'Occurances'})
count=count.sort_values(by='Occurances', axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')

less_than_thirty = count.loc[count['Document Length']<=30,'Occurances'].sum()

print("total records: ", len(raw_data.index))

print("less than 30 characters: ", less_than_thirty)

In [ ]:
# document length  histogram 
color = sns.color_palette()
count =raw_data.length.value_counts()
plt.figure(figsize=(12,6))
sns.barplot(count.index, count.values, alpha=0.8,color=color[3])
plt.xticks(rotation='vertical')
plt.xlabel('Document length', fontsize=12)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.show()